In [1]:
import cv2
from lib.datasets.kitti_utils import *
import torch
import numpy as np
import os

In [16]:
writelist = ['Pedestrian', 'Car', 'Cyclist']
def get_3d_box(objects, get_all_obj = True):
    obj_lst = []
    for obj in objects:
        height = float(obj.box2d[3]) - float(obj.box2d[1]) + 1
        if not get_all_obj:
            if obj.cls_type not in writelist or obj.trucation > 0.5 or obj.occlusion > 2 or height < 25 or obj.trucation == -1:
                continue

        box2d = obj.box2d
        z = obj.generate_corners3d()
        box3d,_ = calib.rect_to_img(z)
        # img_boxes = calib.corners3d_to_img_boxes(z.reshape(-1,8,3))
        obj_lst.append([np.array(box3d), obj.cls_type,obj.box2d,obj.score])
    return obj_lst

def draw_3d_box(obj_lst,img):
    for i in range(obj_lst.shape[0]):
        for j in range(4):
            cv2.line(img, tuple(np.int0(obj_lst[i][0][j])), tuple(np.int0(obj_lst[i][0][(j+1)%4])), (0, 255, 0), 2)
            cv2.line(img, tuple(np.int0(obj_lst[i][0][j+4])), tuple(np.int0(obj_lst[i][0][((j+1)%4)+4])), (0, 255, 0), 2)
            cv2.line(img, tuple(np.int0(obj_lst[i][0][j])), tuple(np.int0(obj_lst[i][0][j+4])), (0, 255, 0), 2)
        
        box_2d = obj_lst[i][2]

        text_pos = np.array([box_2d[0],box_2d[1]-10])
        obj_width = box_2d[2] - box_2d[0]

        # text = "Score: " + str(obj_lst[i][-1])
        text = obj_lst[i][1]
        font = cv2.FONT_HERSHEY_SIMPLEX
        textsize = cv2.getTextSize(text, font, 1, 2)[0]
        textX = int(text_pos[0]+((obj_width - textsize[0]) / 2))
        textY = int(text_pos[1])

        # img = cv2.putText(img,text=text,org = (textX,textY),fontFace=font,fontScale=0.7,color = (36,255,12),thickness=2)
    return img

In [17]:
# visualize detail 1 image
dataset_path = "E:\LVTN\Dataset\KITTI\\training"

filename = "004281"
img_file = os.path.join(dataset_path,"image_2",filename+".png")
label_file = os.path.join(dataset_path,"label_2",filename +".txt")
calib_file = os.path.join(dataset_path,"calib",filename+".txt")


with open(label_file, 'r') as f:
    lines = f.readlines()
    objects = [Object3d(line) for line in lines]

calib = Calibration(calib_file)

box2d_list = []
center2D_list = []
center3D_list = []
box3d_list = []
bev_box_list = []
draw_3d = True
obj_lst = get_3d_box(objects,False)
obj_lst = np.stack(obj_lst)

img = cv2.imread(img_file)

print(len(box3d_list))
if draw_3d:
    img = draw_3d_box(obj_lst,img)

cv2.imshow('image', img)
# path_write = "E:\LVTN\kitti_object_vis\imgs\%06d" % (int(filename))
# if not os.path.exists(path_write):
#     os.mkdir(path_write)
# cv2.imwrite(os.path.join(path_write,"gt.png"),img)
cv2.waitKey(0)
cv2.destroyAllWindows()

c:\Users\phatd\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\shape_base.py:420: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arrays = [asanyarray(arr) for arr in arrays]


0


In [20]:

dataset_path = "E:\LVTN\Dataset\KITTI\\training"
pred_path = "homo_16"
baseline_path="base_16_3class"
filename = "002601"

# additional_path = "homo_caronly"
# additional_path = "baseline_caronly"
# additional_path = "best_eval_author"

pred_file = os.path.join(dataset_path,pred_path,filename +".txt")
baseline_file= os.path.join(dataset_path,baseline_path,filename +".txt")
label_file = os.path.join(dataset_path,"label_2",filename+".txt")
img_file = os.path.join(dataset_path,"image_2",filename+".png")
calib_file = os.path.join(dataset_path,"calib",filename+".txt")

with open(pred_file, 'r') as f:
    lines = f.readlines()
    objects = [Object3d(line) for line in lines]

calib = Calibration(calib_file)

box3d_list = []
score_list = []
draw_3d = True
obj_lst = get_3d_box(objects)
obj_lst = np.stack(obj_lst)

with open(baseline_file, 'r') as f:
    lines = f.readlines()
    objects = [Object3d(line) for line in lines]

obj_lst2 = get_3d_box(objects)
obj_lst2 = np.stack(obj_lst2)

img = cv2.imread(img_file)
img2 = img.copy()
img = draw_3d_box(obj_lst,img)
img2 =  draw_3d_box(obj_lst2,img2)
    
cv2.imshow('pred', img)
cv2.imshow('base', img2)
# path_write = "E:\LVTN\kitti_object_vis\imgs\%06d" % (int(filename))
# if not os.path.exists(path_write):
#     os.mkdir(path_write)
# cv2.imwrite(os.path.join(path_write,"homo_16.png"),img)
# cv2.imwrite(os.path.join(path_write,"base_16.png"),img2)
cv2.waitKey(0)
cv2.destroyAllWindows()